### Algortitmo que realiza las siguientes funciones:
    1. Abrir un fichero Excel (.xls) como si fuera un (.txt), para obtener un (.xml).
    2. Parsear el contenido del (.xml) y obtener toda la información del Excel(todas las celdas).
        2.1. Se obtiene toda la información de cada fila del Excel -> ROW.
        2.2. Se obtiene toda la información de cada celda del Excel -> CELL. 
            2.2.1. Se obtienen los valores MergeDown y MergeAcross de cada celda.
        2.3. Se obtiene la información "visual"  de cada celda -> DATA.
    3. Con toda la información anterior, se crea un fichero (.csv) y se va guardando la información correctamente.
    
#### Autor: Mario de la Parte Izquierdo 
#### Fecha: 31/03/2019

In [20]:
import re # Regular Expressions

In [21]:
f = open("datos.xls","r") # Se abre el fichero original corrupto [r: read(estamos abriendo un archivo para leerlo)].
a = f.read() 
f.close()
#print(a) # Visualizar todo lo que tiene el xml.

In [22]:
nombreDelArchivo = "ficheroBueno.csv" # Variable global que recoge el nombre del archivo .csv que se va a crear.
archivo = open(nombreDelArchivo ,"w")  # 1. Se crear el fichero .csv

posicion = 0
lista_num_ceros_fila_siguiente = []
filaMergeDownMayorQueCero = False
filaEspecial = False
flag = True

datos = re.findall("<Row>(.+?)</Row>", a) # Se obtiene toda la información de cada fila del Excel (ROW)

for i, fila in enumerate(datos): # Se recorre cada fila
    #print(i)
    #print(fila)
    #print("\n")
    # Conclusión: Hay que tratar 21 filas    
    cell = re.findall("<Cell(.+?)</Cell>", fila) # CELL (es una lista de strings)
    #print(cell)
    
    for j, celda in enumerate(cell):
        step_0 = celda.split(' ss:MergeDown="')  # Se saca MergeDown y  MergeAcross de cada celda
        step_1 = step_0[1].split('" ss:MergeAcross="')
        step_2 = step_1[1].split('"')
        #print("MergeDown:")
        #print(step_1[0])
        MergeDown = step_1[0] 
        #print("MergeAcross:")
        #print(step_2[0])
        MergeAcross = step_2[0]
        
        # Se rellena la lista_num_ceros_fila_siguiente 
        if(int(MergeDown) > 0):
            filaMergeDownMayorQueCero = True
        if (filaMergeDownMayorQueCero == True):
            lista_num_ceros_fila_siguiente.append(int(MergeDown)) # Para añadir MergeDown a la lista sea 1 o 0.
            if(int(MergeAcross) > 0):
                aux = int(MergeAcross)
                while (aux > 0):
                    lista_num_ceros_fila_siguiente.append(0) # Para añadir un cero en caso de que haya celdas en blanco
                    aux -= 1
            
        if (filaEspecial == True): # Meter info de lista_num_ceros_fila_siguiente [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1]
            #print("filaEspecial activado")
            
            #if (len(lista_num_ceros_fila_siguiente) >= posicion):# Para limpiar la lista en caso de que haya 2 titulaciones en el mismo Excel.
                #lista_num_ceros_fila_siguiente = []
                #posicion = 0
            #print(len(lista_num_ceros_fila_siguiente))
            while (posicion < len(lista_num_ceros_fila_siguiente)   and flag == True):
                if (lista_num_ceros_fila_siguiente[posicion] == 1): # Hay un 1 en la lisya -> meter ; 
                    #print("mete celda en blanco")
                    archivo.write(";")
                else:                                           # Hay un 0 en la lisya -> meter datos 
                    flag = False
                    #print("mete datos")
                    data = re.findall("<Data(.+?)>(.+?)</Data>", celda) # DATA 
                    #print(data)
                    if(len(data)>0):
                        if ( type(data[0][1]) != int ): # Si no es un entero, se entrecomilla
                            archivo.write('"') # Se añaden comillas para que se posteriormente se pueda abrir el .csv sin problemas por lasa tildes
                            archivo.write(data[0][1])
                            archivo.write('"') # Se añaden comillas para que se posteriormente se pueda abrir el .csv sin problemas por lasa tildes
                            archivo.write(";")
                        else:  # si es un entero, no se entrecomilla
                            print("es un entero") # Aqui no entra nunca 
                            archivo.write(data[0][1])
                            archivo.write(";") # Siguiente columna
                        num = int(MergeAcross)
                        while(num > 0):            
                            archivo.write(";")
                            num -= 1
                    else: 
                        archivo.write(";")
                posicion += 1  
                    
        else:   # Caso Normal         
            # Sacar contenido de la celda (DATA)
            data = re.findall("<Data(.+?)>(.+?)</Data>", celda) # DATA 
            if(len(data)>0):
                #print(data[0][1]) 
                if ( type(data[0][1]) != int ): # Si no es un entero, se entrecomilla
                    archivo.write('"') # Se añaden comillas para que se posteriormente se pueda abrir el .csv sin problemas por lasa tildes
                    archivo.write(data[0][1])
                    archivo.write('"') # Se añaden comillas para que se posteriormente se pueda abrir el .csv sin problemas por lasa tildes
                    archivo.write(";") # Siguiente columna
                else:  # si es un entero, no se entrecomilla
                    print("es un entero") # Aqui no entra nunca 
                    archivo.write(data[0][1])
                    archivo.write(";") # Siguiente columna
                num = int(MergeAcross)
                while(num > 0):            
                    archivo.write(";")
                    num -= 1
            else: # Es una celda en blanco (en data hay [])
                #print("EN BLANCO")
                archivo.write(";")
        flag = True
    
    #print(lista_num_ceros_fila_siguiente)
    # Fin del 2do for que recorre cada fila.
    #print("\n") # Por cada cacho del sublime (21 en total)
    
    if (filaEspecial == True):
        while (posicion < len(lista_num_ceros_fila_siguiente) ):
            archivo.write(";") 
            posicion += 1 
    archivo.write("\n") # Siguiente Fila.
    
    filaEspecial = False
    if(filaMergeDownMayorQueCero == True):
        filaEspecial = True
    filaMergeDownMayorQueCero = False
    
archivo.close()
#print(lista_num_ceros_fila_siguiente) # Son 33 columnas

In [15]:
dato = 2

In [16]:
print(type(dato)) #Ver de que tipo de dato es la variable
if ( type(dato) == int ):
    print("Es un entero")
    
print(dato)

<class 'int'>
Es un entero
2
